<a href="https://colab.research.google.com/github/joaopamaral/tcc_puc_cdbd/blob/master/EDA_TCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise Exploratória dos dados - Reducing Commercial Aviation Fatalities

In [0]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_profiling as pp
import seaborn as sns
import sys
import os
import math

IN_COLAB = 'google.colab' in sys.modules

In [0]:
gdrive_kaggle_dir = '/gdrive/My Drive/Kaggle'
api_gdrive_credential_location = f'{gdrive_kaggle_dir}/kaggle.json'
data_gdrive_dir = f'{gdrive_kaggle_dir}/reducing-commercial-aviation-fatalities'
api_local_credential_location = os.path.join(os.path.expanduser('~'), '.kaggle/kaggle.json')

In [3]:
if IN_COLAB:
    if not os.path.isfile('train.csv.zip') or not os.path.isfile('test.csv.zip'):
        from google.colab import drive
        drive.mount('/gdrive')

        if os.path.isdir(gdrive_kaggle_dir):
            if os.path.isfile(f'{data_gdrive_dir}/test.csv.zip') and os.path.isfile(f'{data_gdrive_dir}/train.csv.zip'):
                !cp '{data_gdrive_dir}/train.csv.zip' .
                !cp '{data_gdrive_dir}/test.csv.zip' .
            else:
                !mkdir -p ~/.kaggle
                !cp '{api_gdrive_credential_location}' '{api_local_credential_location}'

if not os.path.isfile('train.csv.zip') or not os.path.isfile('test.csv.zip'):
    if os.path.isfile(api_local_credential_location):
        !pip -q install kaggle --upgrade
        !kaggle competitions download -c reducing-commercial-aviation-fatalities
        if IN_COLAB and os.path.isdir(gdrive_kaggle_dir):
            !cp train.csv.zip '{data_gdrive_dir}'
            !cp test.csv.zip '{data_gdrive_dir}'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
coluns_types = {
    'crew': 'category',
    'experiment': 'category',
    'seat': 'category',
    'event': 'category'
}

train_df = pd.read_csv('train.csv.zip', dtype = coluns_types)

float64_columns = train_df.select_dtypes(np.float64).columns
train_df[float64_columns] = train_df[float64_columns].astype(np.float16)

In [5]:
train_df.head()

,crew,experiment,time,seat,eeg_fp1,eeg_f7,eeg_f8,eeg_t4,eeg_t6,eeg_t5,eeg_t3,eeg_fp2,eeg_o1,eeg_p3,eeg_pz,eeg_f3,eeg_fz,eeg_f4,eeg_c4,eeg_p4,eeg_poz,eeg_c3,eeg_cz,eeg_o2,ecg,r,gsr,event
0,1,CA,0.011719,1,-5.285156,26.781250,-9.523438,-12.796875,16.718750,33.75000,23.718750,-6.695312,29.234375,24.843750,3.921875,18.453125,1.075195,3.089844,37.375000,17.437500,19.203125,20.593750,-3.951172,14.507812,-4520.0,817.5,388.75,A
1,1,CA,0.015625,1,-2.427734,28.437500,-9.320312,-3.757812,15.968750,30.43750,21.015625,-6.476562,26.640625,24.140625,5.468750,20.515625,2.833984,7.101562,31.171875,19.406250,19.687500,21.359375,1.332031,17.750000,-4520.0,817.5,388.75,A
2,1,CA,0.019531,1,10.671875,30.421875,15.351562,24.718750,16.140625,32.15625,25.437500,-0.088684,28.125000,26.890625,8.578125,23.156250,4.519531,12.703125,-12.015625,19.390625,23.171875,22.406250,1.538086,22.250000,-4520.0,817.5,388.75,A
3,1,CA,0.023438,1,11.453125,25.609375,2.433594,12.414062,20.531250,31.50000,19.140625,-0.256592,30.656250,24.250000,8.640625,19.968750,7.277344,11.460938,18.578125,23.156250,22.640625,19.343750,2.544922,19.000000,-4520.0,817.5,388.75,A
4,1,CA,0.027344,1,7.285156,25.937500,0.113586,5.746094,19.828125,28.75000,20.578125,-1.953125,31.718750,25.156250,12.500000,18.750000,7.308594,8.007812,6.554688,22.750000,22.671875,20.296875,1.699219,22.812500,-4520.0,817.5,388.75,A


In [6]:
train_df.describe(include='all')

,crew,experiment,time,seat,eeg_fp1,eeg_f7,eeg_f8,eeg_t4,eeg_t6,eeg_t5,eeg_t3,eeg_fp2,eeg_o1,eeg_p3,eeg_pz,eeg_f3,eeg_fz,eeg_f4,eeg_c4,eeg_p4,eeg_poz,eeg_c3,eeg_cz,eeg_o2,ecg,r,gsr,event
count,4867421,4867421,4.867421e+06,4867421,4.867421e+06,4.867421e+06,4.867421e+06,4.867421e+06,4.867421e+06,4.867421e+06,4.867421e+06,4.867421e+06,4.867421e+06,4.867421e+06,4.867421e+06,4.867421e+06,4.867421e+06,4.867421e+06,4.867421e+06,4.867421e+06,4.867421e+06,4.867421e+06,4.867421e+06,4.867421e+06,4867421.0,4867421.0,4867421.0,4867421
unique,9,3,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
top,6,DA,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A
freq,552958,1658393,NaN,2433939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2848809
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,3.000259e-03,NaN,-1.361000e+03,-1.581000e+03,-1.644000e+03,-1.517000e+03,-1.221000e+03,-1.266000e+03,-1.280000e+03,-1.393000e+03,-2.888000e+03,-1.227000e+03,-2.876000e+03,-1.353000e+03,-4.064000e+03,-2.334000e+03,-1.212000e+03,-1.228000e+03,-1.229000e+03,-1.230000e+03,-6.965000e+02,-1.176000e+03,-18592.0,482.0,0.0,NaN
25%,NaN,NaN,8.806250e+01,NaN,-9.203125e+00,-8.328125e+00,-8.765625e+00,-7.367188e+00,-6.101562e+00,-6.007812e+00,-6.902344e+00,-9.578125e+00,-6.656250e+00,-6.582031e+00,-7.644531e+00,-9.289062e+00,-8.054688e+00,-9.304688e+00,-7.496094e+00,-6.714844e+00,-6.773438e+00,-7.160156e+00,-7.816406e+00,-6.527344e+00,-2550.0,663.0,524.0,NaN
50%,NaN,NaN,1.768750e+02,NaN,3.818359e-01,4.263306e-02,1.140137e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.894043e-01,0.000000e+00,0.000000e+00,0.000000e+00,1.170654e-01,1.106567e-01,5.667114e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5920.0,743.5,770.0,NaN
75%,NaN,NaN,2.682500e+02,NaN,1.030469e+01,8.750000e+00,9.281250e+00,7.437500e+00,6.175781e+00,6.085938e+00,7.070312e+00,1.063281e+01,6.781250e+00,6.746094e+00,7.808594e+00,9.953125e+00,8.632812e+00,9.773438e+00,7.765625e+00,6.832031e+00,6.906250e+00,7.464844e+00,8.023438e+00,6.613281e+00,10288.0,813.5,1197.0,NaN


In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4867421 entries, 0 to 4867420
Data columns (total 28 columns):
crew          category
experiment    category
time          float16
seat          category
eeg_fp1       float16
eeg_f7        float16
eeg_f8        float16
eeg_t4        float16
eeg_t6        float16
eeg_t5        float16
eeg_t3        float16
eeg_fp2       float16
eeg_o1        float16
eeg_p3        float16
eeg_pz        float16
eeg_f3        float16
eeg_fz        float16
eeg_f4        float16
eeg_c4        float16
eeg_p4        float16
eeg_poz       float16
eeg_c3        float16
eeg_cz        float16
eeg_o2        float16
ecg           float16
r             float16
gsr           float16
event         category
dtypes: category(4), float16(24)
memory usage: 241.4 MB


In [8]:
train_df.isna().sum()

crew          0
experiment    0
time          0
seat          0
eeg_fp1       0
eeg_f7        0
eeg_f8        0
eeg_t4        0
eeg_t6        0
eeg_t5        0
eeg_t3        0
eeg_fp2       0
eeg_o1        0
eeg_p3        0
eeg_pz        0
eeg_f3        0
eeg_fz        0
eeg_f4        0
eeg_c4        0
eeg_p4        0
eeg_poz       0
eeg_c3        0
eeg_cz        0
eeg_o2        0
ecg           0
r             0
gsr           0
event         0
dtype: int64

In [0]:
fig, ax = plt.subplots(1, 4, figsize=(25,5))

train_df.crew.value_counts().sort_index().plot.bar(ax=ax[0], color=sns.color_palette());
train_df.experiment.value_counts().sort_index().plot.bar(ax=ax[1], color=sns.color_palette());
train_df.event.value_counts().sort_index().plot.bar(ax=ax[2], color=sns.color_palette());
train_df.seat.value_counts().sort_index().plot.bar(ax=ax[3], color=sns.color_palette());

In [0]:
fig, ax = plt.subplots(1, 3, figsize=(25,5))

train_df.groupby('crew').experiment.value_counts().unstack(0).sort_index().plot.bar(ax=ax[0]);
train_df.groupby('event').experiment.value_counts().unstack(0).sort_index().plot.bar(ax=ax[1]);
train_df.groupby('seat').experiment.value_counts().unstack(0).sort_index().plot.bar(ax=ax[2]);

In [0]:
def plot_eeg(crew='1', experiment='CA', start_time=.5, end_time=.6):
    eeg_cols = train_df.columns[train_df.columns.str.contains('eeg_')]
    fig, ax = plt.subplots(math.ceil(eeg_cols.size / 4), 4, figsize=(20, 15))
    c = sns.color_palette("hls", eeg_cols.size)
    for ix, col in enumerate(eeg_cols):
        train_df.loc[(train_df.crew == crew) & (train_df.experiment == experiment) & 
                     train_df.time.between(start_time, end_time), ['time', col]].set_index('time').sort_index().plot(ax=ax[ix // 4, ix % 4], color=c[ix], legend=True)

plot_eeg()

In [0]:
def plot_ecg(crew='1', experiment='CA', start_time=.5, end_time=.55):
    train_df.loc[(train_df.crew == crew) & (train_df.experiment == experiment) & 
                     train_df.time.between(start_time, end_time), ['time', 'ecg']].set_index('time').sort_index().plot(legend=True);

plot_ecg()

In [0]:
train_df.loc[:, train_df.columns.str.contains('eeg')].plot.box(figsize=(30,7));

In [0]:
fig, ax = plt.subplots(1, 3, figsize=(25,5))

train_df['ecg'].plot.box(ax=ax[0]);
train_df['r'].plot.box(ax=ax[1]);
train_df['gsr'].plot.box(ax=ax[2]);

In [0]:
# pp.ProfileReport(train_df).to_file("report-reducing-commercial-aviation-fatalities.html")

In [0]:
train_df = train_df.drop_duplicates()

![alt text](https://scikit-learn.org/stable/_static/ml_map.png)

In [0]:
coluns_types = {
    'crew': 'category',
    'experiment': 'category',
    'seat': 'category',
    'event': 'category',
    'id': np.int16
}

test_df = pd.read_csv('test.csv.zip', dtype = coluns_types)

float64_columns = test_df.select_dtypes(np.float64).columns
test_df[float64_columns] = test_df[float64_columns].astype(np.float16)

In [22]:
test_df.head()

,id,crew,experiment,time,seat,eeg_fp1,eeg_f7,eeg_f8,eeg_t4,eeg_t6,eeg_t5,eeg_t3,eeg_fp2,eeg_o1,eeg_p3,eeg_pz,eeg_f3,eeg_fz,eeg_f4,eeg_c4,eeg_p4,eeg_poz,eeg_c3,eeg_cz,eeg_o2,ecg,r,gsr
0,0,1,LOFT,0.000000,0,17.906250,6.128906,0.994629,-28.203125,-47.687500,-187.125000,-33.187500,-4.222656,8.179688,33.156250,33.812500,21.750000,16.296875,-7.042969,-14.406250,-4.035156,-0.393799,31.843750,17.078125,-8.140625,-7324.0,643.0,595.00
1,1,1,LOFT,0.000000,1,45.875000,94.750000,23.296875,1.391602,2.060547,-5.144531,6.394531,33.406250,-295.000000,8.375000,29.640625,16.218750,850.500000,19.890625,-215.125000,2.119141,7.339844,9.664062,1169.000000,-0.024414,-3336.0,826.5,136.25
2,2,1,LOFT,0.003906,0,33.125000,28.359375,-7.238281,-7.691406,-25.828125,-107.250000,12.843750,1.214844,13.320312,44.437500,37.593750,29.078125,22.312500,-7.640625,-10.367188,10.953125,6.511719,36.031250,24.359375,0.956055,-7324.0,643.0,595.00
3,3,1,LOFT,0.003906,1,43.281250,95.875000,18.703125,-1.432617,-4.234375,-8.023438,7.425781,27.343750,-310.250000,5.390625,27.734375,7.218750,816.500000,13.828125,-214.250000,-4.914062,3.503906,3.878906,1159.000000,-0.047607,-3336.0,826.5,136.25
4,4,1,LOFT,0.007812,0,7.929688,3.460938,-10.859375,-26.359375,-25.890625,37.000000,-50.343750,-11.679688,-1.143555,-17.593750,34.062500,22.906250,19.031250,2.044922,-20.781250,-3.613281,-7.539062,3.083984,13.648438,1.473633,-7324.0,643.0,595.00


In [26]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17965143 entries, 0 to 17965142
Data columns (total 28 columns):
id            int16
crew          category
experiment    category
time          float16
seat          category
eeg_fp1       float16
eeg_f7        float16
eeg_f8        float16
eeg_t4        float16
eeg_t6        float16
eeg_t5        float16
eeg_t3        float16
eeg_fp2       float16
eeg_o1        float16
eeg_p3        float16
eeg_pz        float16
eeg_f3        float16
eeg_fz        float16
eeg_f4        float16
eeg_c4        float16
eeg_p4        float16
eeg_poz       float16
eeg_c3        float16
eeg_cz        float16
eeg_o2        float16
ecg           float16
r             float16
gsr           float16
dtypes: category(3), float16(24), int16(1)
memory usage: 908.0 MB


In [27]:
test_df.isna().sum()

id            0
crew          0
experiment    0
time          0
seat          0
eeg_fp1       0
eeg_f7        0
eeg_f8        0
eeg_t4        0
eeg_t6        0
eeg_t5        0
eeg_t3        0
eeg_fp2       0
eeg_o1        0
eeg_p3        0
eeg_pz        0
eeg_f3        0
eeg_fz        0
eeg_f4        0
eeg_c4        0
eeg_p4        0
eeg_poz       0
eeg_c3        0
eeg_cz        0
eeg_o2        0
ecg           0
r             0
gsr           0
dtype: int64

In [0]:
from sklearn.linear_model import SGDClassifier

In [0]:
sgd_model = SGDClassifier()
sgd_model.fit